In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [4]:
data = pd.read_csv('./dataset/credit-a.csv', header=None)

In [5]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,30.83,0.000,0,0,9,0,1.25,0,0,1,1,0,202,0.0,-1
1,1,58.67,4.460,0,0,8,1,3.04,0,0,6,1,0,43,560.0,-1
2,1,24.50,0.500,0,0,8,1,1.50,0,1,0,1,0,280,824.0,-1
3,0,27.83,1.540,0,0,9,0,3.75,0,0,5,0,0,100,3.0,-1
4,0,20.17,5.625,0,0,9,0,1.71,0,1,0,1,2,120,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,21.08,10.085,1,1,11,1,1.25,1,1,0,1,0,260,0.0,1
649,1,22.67,0.750,0,0,0,0,2.00,1,0,2,0,0,200,394.0,1
650,1,25.25,13.500,1,1,13,7,2.00,1,0,1,0,0,200,1.0,1
651,0,17.92,0.205,0,0,12,0,0.04,1,1,0,1,0,280,750.0,1


In [6]:
X = data.iloc[:, :-1]

In [7]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,30.83,0.000,0,0,9,0,1.25,0,0,1,1,0,202,0.0
1,1,58.67,4.460,0,0,8,1,3.04,0,0,6,1,0,43,560.0
2,1,24.50,0.500,0,0,8,1,1.50,0,1,0,1,0,280,824.0
3,0,27.83,1.540,0,0,9,0,3.75,0,0,5,0,0,100,3.0
4,0,20.17,5.625,0,0,9,0,1.71,0,1,0,1,2,120,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,21.08,10.085,1,1,11,1,1.25,1,1,0,1,0,260,0.0
649,1,22.67,0.750,0,0,0,0,2.00,1,0,2,0,0,200,394.0
650,1,25.25,13.500,1,1,13,7,2.00,1,0,1,0,0,200,1.0
651,0,17.92,0.205,0,0,12,0,0.04,1,1,0,1,0,280,750.0


In [8]:
Y = data.iloc[:, -1]
Y

0     -1
1     -1
2     -1
3     -1
4     -1
      ..
648    1
649    1
650    1
651    1
652    1
Name: 15, Length: 653, dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653 entries, 0 to 652
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       653 non-null    int64  
 1   1       653 non-null    float64
 2   2       653 non-null    float64
 3   3       653 non-null    int64  
 4   4       653 non-null    int64  
 5   5       653 non-null    int64  
 6   6       653 non-null    int64  
 7   7       653 non-null    float64
 8   8       653 non-null    int64  
 9   9       653 non-null    int64  
 10  10      653 non-null    int64  
 11  11      653 non-null    int64  
 12  12      653 non-null    int64  
 13  13      653 non-null    int64  
 14  14      653 non-null    float64
 15  15      653 non-null    int64  
dtypes: float64(4), int64(12)
memory usage: 81.8 KB


In [10]:
# 把-1改成0, 表示负样本. 
Y.replace(-1, 0, inplace=True)

In [11]:
Y.unique()

array([0, 1], dtype=int64)

In [12]:
Y.value_counts()

1    357
0    296
Name: 15, dtype: int64

In [13]:
# 变成tensor
X = torch.from_numpy(X.values).type(torch.FloatTensor)
Y = torch.from_numpy(Y.values.reshape(-1, 1)).type(torch.FloatTensor)

In [15]:
X.shape

torch.Size([653, 15])

In [14]:
from torch import nn

In [16]:
model = nn.Sequential(
    nn.Linear(15, 512),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

In [17]:
model

Sequential(
  (0): Linear(in_features=15, out_features=512, bias=True)
  (1): Linear(in_features=512, out_features=1, bias=True)
  (2): Sigmoid()
)

In [18]:
# 二分类交叉熵
loss_fn = nn.BCELoss()

In [19]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [20]:
# 正常神经网络训练是一个batch一个batch
batch_size = 32
steps = 653 // 32

In [21]:
for epoch in range(1000):
    for batch in range(steps):
        start = batch * batch_size
        end = start + batch_size
        x = X[start:end]
        y = Y[start:end]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [23]:
# 训练好的权重参数
model.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.2510, -0.0917, -0.1623,  ...,  0.0688,  0.0101,  0.2397],
                      [-0.0924, -0.0720,  0.2507,  ...,  0.0427,  0.1288,  0.0303],
                      [ 0.0991, -0.0459, -0.0488,  ..., -0.2309, -0.0752, -0.0264],
                      ...,
                      [-0.2204,  0.0958,  0.0440,  ...,  0.2151,  0.1661, -0.2102],
                      [-0.1011,  0.1929, -0.0635,  ...,  0.1204,  0.1412,  0.1342],
                      [-0.2260,  0.0124,  0.1983,  ...,  0.1256, -0.2107,  0.2492]])),
             ('0.bias',
              tensor([-0.1556,  0.0715, -0.2211,  0.1314,  0.0178,  0.1937,  0.0756,  0.1372,
                       0.1291,  0.1450, -0.1565,  0.2017, -0.2572,  0.0980, -0.0820,  0.0146,
                       0.1368,  0.1496,  0.1936,  0.0957, -0.1675,  0.2450,  0.1044, -0.1162,
                       0.1013,  0.0354,  0.0818,  0.0776, -0.0998,  0.2017, -0.2064,  0.0156,
                       0.2510, -0.0912, 

In [30]:
# pytorch要自己写的东西, 比tensorflow多一些. 
# 比如计算准确率需要自己算.
# 设定阈值, 认为>= 0.5是正样本
((model(X).data.numpy() >= 0.6) == Y.numpy()).mean()

0.5911179173047473